In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from plotly import graph_objects as go
from plotly import colors
from plotly.colors import sample_colorscale

In [2]:
import sys
import os

# Add the parent directory to sys.path
sys.path.append(os.path.abspath('..'))

In [3]:
from src.table import CherryTable
import src.utils as utils
import src.preprocessing as pre

In [4]:
train, test, mapping_dict = pre.separate_year(planting_meta_path='../data/planting_meta.json', weekly_summary_path='../data/weekly_summary.csv')

In [5]:
model =utils.train_harvest_model(train)

Epoch [1/5], Loss: 869098.8096
Epoch [2/5], Loss: 799709.9590
Epoch [3/5], Loss: 792688.7305
Epoch [4/5], Loss: 762202.8954
Epoch [5/5], Loss: 710263.5054


In [6]:
predictions = utils.predict_harvest(model, test)

In [7]:
meta = pre.decode(test, mapping_dict)
actuals = test.Y_kilos.detach().numpy()

In [8]:
table = CherryTable(meta, {'predictions':predictions}, actuals)

In [9]:
a,d,ha_info = table.graph_ready(ranches=True,classes=True)

In [10]:
title = list(d.keys())[0]

In [11]:
d[title]

0          1           2           3            4   \
Ranch Class                                                              
OAP   BSUF   -5.484486  -9.888428 -145.414291   61.635513   598.579346   
      CHE     2.456810   7.943494 -110.876961   25.598200   533.327026   
SFB   BSUF   -0.968893  -2.584510  -53.231823   20.548948   220.215179   
      CHE     6.975963  14.065204  -56.760666   -1.648409   309.744110   
SGB   BSUF   -1.744256  -3.504397  -77.444740   29.990992   324.569458   
      CHE     2.090929   5.061502  -56.999886   11.244286   276.072754   
SJB   BSUF   -0.740367  -1.686371  -42.077534   15.933920   177.131393   
      CHE     4.001469   8.502715  -49.061676    3.778631   253.577530   
SMB   BSUF   -2.673324  -6.069169  -90.310112   37.012173   369.729218   
      CHE     3.249020   6.576526  -53.011383    7.103358   263.437744   
VAP   BSUF   -3.271985 -19.860682 -370.617340  144.046890  1503.036499   
      CHE    17.622568  26.072376 -225.247696   32.219738  1077.469604   

                    5          6           7           8            9   \
Ranch Class                                                              
OAP   BSUF   -2.569424 -27.746550   72.740891  107.973282  1810.401489   
      CHE   -29.910587 -29.806204   32.443451   15.004826  1266.989624   
SFB   BSUF   -1.542894 -10.530596   25.854286   37.041763   656.682251   
      CHE   -28.174015 -18.226120    5.046189  -27.609840   585.230957   
SGB   BSUF   -2.857521 -13.151806   37.348942   52.098984   953.347168   
      CHE   -16.086643 -13.631971   15.750303    4.529050   640.737610   
SJB   BSUF   -2.021791  -7.685522   19.836365   26.772032   513.898499   
      CHE   -18.968668 -12.723200    9.334312  -11.505804   524.174683   
SMB   BSUF   -1.681117 -17.151976   44.276539   63.896141  1109.295410   
      CHE   -17.511181 -14.336872   12.091917   -5.027208   577.258484   
VAP   BSUF   -1.477325 -65.628906  190.569489  283.284485  4609.600586   
      CHE   -58.762928 -54.617683   68.563141   25.552013  2556.627686   

                       10            11            12            13  \
Ranch Class                                                           
OAP   BSUF    4816.266602  11186.562500  15852.775391  20770.382812   
      CHE     3784.985596   8272.390625  12201.799805  16853.367188   
SFB   BSUF    1755.427002   4065.805908   5771.964355   7585.699707   
      CHE     1967.473755   4055.883789   6221.390137   9020.082031   
SGB   BSUF    2565.803467   5925.062988   8428.082031  11104.390625   
      CHE     1934.303589   4207.979980   6225.786621   8638.094727   
SJB   BSUF    1391.111816   3202.841553   4565.090820   6032.451660   
      CHE     1677.529663   3545.687500   5348.451660   7602.751953   
SMB   BSUF    2960.208008   6865.385254   9738.205078  12782.999023   
      CHE     1793.260620   3843.761963   5743.079590   8068.996094   
VAP   BSUF   12151.635742  28357.263672  40056.914062  52305.949219   
      CHE     7597.104492  16642.242188  24503.505859  33826.410156   

                       14            15            16            17  \
Ranch Class                                                           
OAP   BSUF   24056.412109  23987.316406  22440.328125  19080.250000   
      CHE    19649.812500  19700.138672  18235.365234  15363.193359   
SFB   BSUF    8789.068359   8766.237305   8197.151367   6965.821777   
      CHE    10576.403320  10653.318359   9774.263672   8169.068848   
SGB   BSUF   12867.471680  12839.718750  11997.331055  10194.018555   
      CHE    10073.312500  10105.269531   9345.362305   7870.705078   
SJB   BSUF    6992.543457   6979.362305   6518.500488   5535.058594   
      CHE     8889.905273   8939.452148   8231.437500   6904.257324   
SMB   BSUF   14805.984375  14767.655273  13810.643555  11739.398438   
      CHE     9423.833984   9465.502930   8734.083008   7338.564941   
VAP   BSUF   60546.554688  60344.386719  56509.117188  48086.320312   
      CHE    39427.253906  3

In [12]:
df = d[title].T

In [13]:
af = d['actuals_summed'].T

In [14]:
af

Ranch      OAP               SFB              SGB             SJB          \
Class     BSUF      CHE     BSUF     CHE     BSUF     CHE    BSUF     CHE   
0          0.0      0.0      0.0     0.0      0.0     0.0     0.0     0.0   
1          0.0      0.0      0.0     0.0      0.0     0.0     0.0     0.0   
2          0.0      0.0      0.0     0.0      0.0     0.0     0.0     0.0   
3          0.0      0.0      0.0     0.0      0.0     0.0     0.0     0.0   
4          0.0      0.0      0.0     0.0      0.0     0.0     0.0     0.0   
5          0.0      0.0    129.0   213.0      0.0     0.0     0.0     0.0   
6        247.0      0.0      0.0  1333.0      0.0     0.0     0.0     0.0   
7          0.0      0.0    852.0   883.0     33.0     0.0     0.0     0.0   
8          0.0    796.0   2724.0  2004.0    116.0    36.0  1482.0   507.0   
9       1280.0   2267.0      0.0     0.0    307.0   108.0  4236.0   742.0   
10      7385.0   8092.0   1261.0   470.0   1513.0   471.0  5472.0  1671.0   
11     18731.0  12703.0    726.0   624.0   2223.0  1097.0  7925.0  5822.0   
12     25271.0  21517.0   3146.0  1476.0   6290.0  2079.0  8861.0  6039.0   
13     25938.0  20780.0   7817.0  1531.0   5475.0  5314.0  6686.0  3924.0   
14     30949.0  20929.0  10482.0  4268.0  13021.0  6126.0  7671.0  2597.0   
15     32539.0  15468.0  19272.0  6402.0   8873.0  7149.0  8114.0  4681.0   
16     22353.0  18699.0  14579.0  6340.0   9305.0  5984.0  4190.0  3747.0   
17     15831.0  13833.0  10325.0  4926.0  10036.0  6390.0  3624.0  3036.0   
18     21066.0   8567.0   4596.0  6382.0   1758.0  2642.0  1331.0  2483.0   
19     12352.0   4687.0   2564.0  2066.0   2074.0   832.0   460.0  1238.0   

Ranch      SMB               VAP           
Class     BSUF      CHE     BSUF      CHE  
0          0.0      0.0      0.0      0.0  
1          0.0      0.0      0.0      0.0  
2          0.0      0.0      0.0      0.0  
3          0.0      0.0      0.0      0.0  
4          0.0      0.0      0.0      0.0  
5          0.0      0.0      0.0      0.0  
6          0.0      0.0      0.0      0.0  
7          0.0      0.0      0.0      0.0  
8        480.0      0.0     24.0      0.0  
9          0.0   1023.0    587.0     40.0  
10      3236.0   3443.0   4292.0   2283.0  
11      6670.0   5855.0  10141.0   3014.0  
12     13066.0   7161.0  11268.0   8268.0  
13     14370.0   8003.0  27651.0  12951.0  
14     11039.0  10300.0  57526.0  21938.0  
15      5944.0   6524.0  28505.0  23379.0  
16      2399.0   2888.0  35783.0  28990.0  
17       710.0   1185.0  35544.0  30833.0  
18       869.0   2180.0  39197.0  33812.0  
19       715.0   2236.0  41522.0  30767.0

In [15]:
fig = go.Figure()
x_values = df.index
colors = sample_colorscale('Viridis', [i / (len(df.columns) - 1) for i in range(len(df.columns))])
color_map = {key: colors[i] for i, key in enumerate(df.columns)}

for i, col in enumerate(df.columns):
    label = col[0] + ': ' + col[1]
    fig.add_trace(go.Scatter(x=x_values, y=df[col], name=label + ' Predicted', line=dict(color=color_map[col])))
    fig.add_trace(go.Bar(x=x_values, y=af[col], name= label + ' Actual', marker_color=color_map[col], opacity=0.75))

visibility_matrix = np.eye(2* len(df.columns), dtype=bool).repeat(2, axis=1)

buttons = [dict(label=f"{key[0]}: {key[1]}",method='update',args=[{'visible': visibility_matrix[i].tolist()}, {'title': f'Selected: {key[0]}: {key[1]}'}]) for i,key in enumerate(df.columns)]


# Add "Show All" button
buttons.append(dict(
    label='Show All', 
    method='update',
    args=[{'visible': [True]*(len(df.columns)*2)}, {'title': 'All Traces Visible'}]
))

# Add dropdown menu to layout
fig.update_layout(
    updatemenus=[dict(
        active=0,
        buttons=buttons,
        direction='down',
        showactive=True,
        x=1.05,
        xanchor='left',
        y=1,
        yanchor='top'
    )],
    title=title + ' Trace Filter with Dropdown',
    template='plotly_white',
    xaxis_title='Week after Transplant',
    yaxis_title='Harvest (kg)'
)

fig.show()


In [16]:
import src.curve_alter as curve_alter

In [17]:
k = 10

In [18]:
actuals = af.T.values[0][:k]

In [19]:
preds =df.T.values[0]

array([-1.10778678e+05,  2.34142721e+07,  1.42642592e+08, -1.99106100e+10,
       -1.22840958e+09,  1.90660661e+11,  7.18414943e+12, -1.53270185e+14,
        3.69369398e+16, -1.41234620e+18,  4.71489403e+19, -9.60340455e+20,
        1.80846061e+22, -3.01050978e+23,  4.31454980e+24, -5.80133119e+25,
        7.08968847e+26, -8.13908194e+27,  6.36997210e+28])